Libraries

In [24]:
#libraries
import pandas as pd
import re
from bs4 import BeautifulSoup
import contractions
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AAKASH\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\AAKASH\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AAKASH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AAKASH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AAKASH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Reading File

In [26]:
#reading the csv
df = pd.read_csv(r'C:\Users\AAKASH\OneDrive\Documents\internship\Emotions_training.csv')
df.head()

Converting into Lower case

In [28]:
#converting text into lower case
df['lw_content'] = df['text'].apply(lambda x: x.lower())

Removing non textual content

In [29]:
#removing non textual content
def remove_non_textual(text):
    return re.sub(r'[^\w\s]', '', text)
df['nt_content'] = df['lw_content'].apply(remove_non_textual)


Removing Links

In [30]:
# to strip HTML tags 
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()
df['nonhtml_text'] = df['nt_content'].apply(remove_html_tags)

Removing Extra spaces

In [31]:
# to expand contractions 
df['ct_data'] = df['nonhtml_text'].apply(lambda x: contractions.fix(x))

#Removing stop words

In [33]:
#removing stop words
stopwords.words('english')
[punc for punc in string.punctuation]
def text_process(msg):
  nopunc= [char for char in msg if char not in string.punctuation]
  nopunc= ''.join(nopunc)
  return ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])
df['cleaned_data']= df['ct_data'].apply(text_process)

Lemmatizing of data

In [34]:
#lemmatizing of data
lemmatizer = WordNetLemmatizer()
def lemmatize_sentence(sentence):
    tokens = nltk.word_tokenize(sentence)  
    lemmatized_sentence = ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens])
    return lemmatized_sentence
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()  
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) 
df['lemmatized_text'] = df['cleaned_data'].apply(lemmatize_sentence)

In [36]:
df['lemmatized_text'].head()

0                                        feel humiliate
1     go feel hopeless damn hopeful around someone c...
2                    grab minute post feel greedy wrong
3     ever feel nostalgic fireplace know still property
4                                          feel grouchy
                            ...                        
95         feel like throw away shitty piece shit paper
96    start feel wryly amuse banal comedy error life...
97    find every body beautiful want people feel vit...
98    hear owner feel victimize association associat...
99    say goodbye fam sad cry feel like heartless bi...
Name: lemmatized_text, Length: 100, dtype: object

Convert the Text corpus to a matrix of word counts(tf-IDF)

In [40]:
def tfidf(data, column):
    vectorizer = TfidfVectorizer() 
    return vectorizer.fit_transform(data[column])

tfidf_matrix = tfidf(df, 'text')
tfidf_matrix_array = tfidf_matrix.toarray()
print("TF-IDF Matrix:")
print(tfidf_matrix_array)
print("Shape of TF-IDF matrix:", vectors.shape)

TF-IDF Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of TF-IDF matrix: (16000, 15186)


In [ ]:
Dividing the dataset in to Train (70%), Test (20%) and Validation (10%) datasets.

In [41]:
np.random.shuffle(tfidf_matrix_array)
train_size = int(0.7 * len(tfidf_matrix_array))
test_size = int(0.2 * len(tfidf_matrix_array))
val_size = len(tfidf_matrix_array) - train_size - test_size

# Split the data
train_data = tfidf_matrix_array[:train_size]
test_data = tfidf_matrix_array[train_size:train_size+test_size]
val_data = tfidf_matrix_array[train_size+test_size:]

print("Train data size:", len(train_data))
print("Test data size:", len(test_data))
print("Validation data size:", len(val_data))


Train data size: 11200
Test data size: 3200
Validation data size: 1600
